#  基于MindSpore实现LSTM算法 
本实验基于MindSpore构建LSTM模型，使用SentimentNet实现情感分类,并训练和测试模型。
## 1 实验目的
1.通过实验了解LSTM算法

2.基于MindSpore中实现LSTM算法
## 2 LSTM算法原理介绍
LSTM四个函数层与具体介绍如下：

(1)第一个函数层：遗忘门
    ![jupyter](./Figures/fig001.png)
 
对于上一时刻LSTM中的单元状态，一些信息可能会随着时间的流逝而过时。为了不让过多记忆影响神经网络对现在输入的处理，我们应该选择性遗忘一些在之前单元状态中的分量——这个工作就交给了遗忘门。

每一次输入一个新的输入，LSTM会先根据新的输入和上一时刻的输出决定遗忘之前的哪些记忆——输入和上一步的输出会整合为一个单独的向量，然后通过sigmoid神经层，最后点对点的乘在单元状态上。因为sigmoid 函数会将任意输入压缩到 (0,1) 的区间上，我们可以非常直觉的得出这个门的工作原理 —— 如果整合后的向量某个分量在通过sigmoid层后变为0，那么显然单元状态在对位相乘后对应的分量也会变成0，换句话说，遗忘了这个分量上的信息；如果某个分量通过sigmoid层后为1，单元状态会“保持完整记忆”。不同的sigmoid输出会带来不同信息的记忆与遗忘。通过这种方式，LSTM可以长期记忆重要信息，并且记忆可以随着输入进行动态调整。下面的公式可以用来描述遗忘门的计算，其中f_t就是sigmoid神经层的输出向量：

$f_t=σ(W_f∙[h_(t-1),x_t ]+b_f)$

（2）第二个、第三个函数层：记忆门
记忆门是用来控制是否将在t时刻（现在）的数据并入单元状态中的控制单位。首先，用tanh函数层将现在的向量中的有效信息提取出来，然后使用sigmoid函数来控制这些记忆要放多少进入单元状态。这两者结合起来就可以做到：

 ![jupyter](./Figures/fig002.png)
 
从当前输入中提取有效信息；对提取的有效信息做出筛选，为每个分量做出评级(0 ~ 1)，评级越高的最后会有越多的记忆进入单元状态。下面的公式可以分别表示这两个步骤在LSTM中的计算：

$C_{t}^{'}=tanh⁡(W_c∙[h_{(t-1)},x_t ]+b_c)$

$i_t=σ(W_i∙[h_{(t-1)},x_t ]+b_i)$

（3）第四个函数层：输出门

输出门就是LSTM单元用于计算当前时刻的输出值的神经层。输出层会先将当前输入值与上一时刻输出值整合后的向量用sigmoid函数提取其中的信息，然后，会将当前的单元状态通过tanh函数压缩映射到区间(-1, 1)中，将经过tanh函数处理后的单元状态与sigmoid函数处理后的单元状态，整合后的向量点对点的乘起来就可以得到LSTM在 t时刻的输出。

LSTM模型是由时刻的输入词$X_{t}$ ，细胞状态$C_{t}$，临时细胞状态$\widetilde{C_{t}} $，隐层状态$h_{t}$，遗忘门$f_{t}$，记忆门$i_{t}$，输出门$ o_{t}$组成。LSTM的计算过程可以概括为:通过对细胞状态中信息遗忘和记忆新的信息使得对后续时刻计算有用的信息得以传递，而无用的信息被丢弃，并在每个时间步都会输出隐层状态$h_{t}$ ，其中遗忘、记忆与输出由通过上个时刻的隐层状态$h_{t-1}$和当前输入$X_{t}$计算出来的遗忘门$f_{t}$，记忆门$ i_{t}$，输出门$o_{t}$来控制。

LSTM总体框架图如下：
 ![jupyter](./Figures/fig003.png)


## 3 实验环境
### 实验环境要求
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|
## 4 数据处理
IMDB是一个与国内豆瓣比较类似的与电影相关的网站，而本次实验用到的数据集是这个网站中的一些用户评论。IMDB数据集共包含50000项影评文字，训练数据和测试数据各25000项，每一项影评文字都被标记为正面评价或负面评价，所以本实验可以看做一个二分类问题。IMDB数据集官网：[Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/)。

方式一，从斯坦福大学官网下载aclImdb_v1.tar.gz并解压。

方式二，从华为云OBS中下载aclImdb_v1.tar.gz并解压。

同时，我们要下载GloVe文件，并在文件glove.6B.300d.txt开头处添加新的一行400000 300，即总共读取400000个单词，每个单词用300维度的词向量表示。 修改glove.6B.300.txt如下:

400000 300

the -0.071549 0.093459 0.023738 -0.090339 0.056123 0.32547…

确定评价标准：
作为典型的分类问题，情感分类的评价标准可以比照普通的分类问题处理。常见的精度（Accuracy）、精准度（Precision）、召回率（Recall）和F_beta分数都可以作为参考。

精度（Accuracy）=分类正确的样本数目/总样本数目

精准度（Precision）=真阳性样本数目/所有预测类别为阳性的样本数目

召回率（Recall）=真阳性样本数目/所有真实类别为阳性的样本数目

F1分数=(2∗Precision∗Recall)/(Precision+Recall)

在IMDB这个数据集中，正负样本数差别不大，可以简单地用精度（accuracy）作为分类器的衡量标准。

## 5 模型构建
（1）导入Python库&模块并配置运行信息

导入MindSpore模块和辅助模块。

代码如下：

In [1]:
#导入mindspore模块和辅助模块
import argparse
from mindspore import context
from easydict import EasyDict as edict
from itertools import chain
import numpy as np
import gensim
from mindspore.mindrecord import FileWriter
import os
import mindspore.dataset as ds
import math
from mindspore import Tensor, nn, context, Parameter, ParameterTuple
from mindspore.common.initializer import initializer
import mindspore.ops as ops
from mindspore.train import Model,CheckpointConfig, ModelCheckpoint, TimeMonitor, LossMonitor,Accuracy
from mindspore import nn
from mindspore import load_checkpoint, load_param_into_net

(2）定义参数变量

device_target：指定Ascend或CPU/GPU环境。
pre_trained：预加载CheckPoint文件。
preprocess：是否预处理数据集，默认为否。
aclimdb_path：数据集存放路径。
glove_path：GloVe文件存放路径。
preprocess_path：预处理数据集的结果文件夹。
ckpt_path：CheckPoint文件路径。

In [4]:
import argparse
from mindspore import context
from easydict import EasyDict as edict
# LSTM网络参数
lstm_cfg = edict({
    'num_classes': 2,
    'learning_rate': 0.1,
    'momentum': 0.9,
    'num_epochs': 10,
    'batch_size': 64,
    'embed_size': 300,
    'num_hiddens': 100,
    'num_layers': 2,
    'bidirectional': True,
    'save_checkpoint_steps': 390,
    'keep_checkpoint_max': 10
})
cfg = lstm_cfg
parser = argparse.ArgumentParser(description='MindSpore LSTM Example')
parser.add_argument('--preprocess', type=str, default='false', choices=['true', 'false'],
                    help='whether to preprocess data.')
parser.add_argument('--aclimdb_path', type=str, default="./datasets/aclImdb",
                    help='path where the dataset is stored.')
parser.add_argument('--glove_path', type=str, default="./datasets/glove",
                    help='path where the GloVe is stored.')
parser.add_argument('--preprocess_path', type=str, default="./preprocess",
                    help='path where the pre-process data is stored.')
parser.add_argument('--ckpt_path', type=str, default="./models/ckpt/nlp_application",
                    help='the path to save the checkpoint file.')
parser.add_argument('--pre_trained', type=str, default=None,
                    help='the pretrained checkpoint file path.')
parser.add_argument('--device_target', type=str, default="GPU", choices=['GPU', 'CPU'],
                    help='the target device to run, support "GPU", "CPU". Default: "GPU".')
args = parser.parse_args(['--device_target', 'GPU', '--preprocess', 'true'])
context.set_context(
        mode=context.GRAPH_MODE,
        save_graphs=False,
        device_target=args.device_target)
print("Current context loaded:\n    mode: {}\n    device_target: {}".format(context.get_context("mode"), context.get_context("device_target")))

Current context loaded:
    mode: 0
    device_target: GPU


（3）数据的读取与处理

按照下面的流程解析原始数据集，获得features与labels：sentence->tokenized->encoded->padding->features。

定义convert_to_mindrecord函数将数据集格式转换为MindRecord格式，便于MindSpore读取。
对文本数据集进行处理，包括编码、分词、对齐、处理GloVe原始数据，使之能够适应网络结构。

定义convert_to_mindrecord函数将数据集格式转换为MindRecord格式，便于MindSpore读取。

转换成功后会在`preprocess`目录下生成MindRecord文件，通常该操作在数据集不变的情况下，无需每次训练都执行，此时查看`preprocess`文件目录结构。

```text
preprocess
├── aclImdb_test.mindrecord0
├── aclImdb_test.mindrecord0.db
├── aclImdb_test.mindrecord1
├── aclImdb_test.mindrecord1.db
├── aclImdb_test.mindrecord2
├── aclImdb_test.mindrecord2.db
├── aclImdb_test.mindrecord3
├── aclImdb_test.mindrecord3.db
├── aclImdb_train.mindrecord0
├── aclImdb_train.mindrecord0.db
├── aclImdb_train.mindrecord1
├── aclImdb_train.mindrecord1.db
├── aclImdb_train.mindrecord2
├── aclImdb_train.mindrecord2.db
├── aclImdb_train.mindrecord3
├── aclImdb_train.mindrecord3.db
└── weight.txt
```

In [6]:
import os
from itertools import chain
import numpy as np
import gensim
from mindspore.mindrecord import FileWriter
class ImdbParser():
# 按照下面的流程解析原始数据集，获得features与labels：sentence->tokenized->encoded->padding->features
    def __init__(self, imdb_path, glove_path, embed_size=300):
        self.__segs = ['train', 'test']
        self.__label_dic = {'pos': 1, 'neg': 0}
        self.__imdb_path = imdb_path
        self.__glove_dim = embed_size
        self.__glove_file = os.path.join(glove_path, 'glove.6B.' + str(self.__glove_dim) + 'd.txt')
        self.__imdb_datas = {}
        self.__features = {}
        self.__labels = {}
        self.__vacab = {}
        self.__word2idx = {}
        self.__weight_np = {}
        self.__wvmodel = None
    def parse(self):
#解析imdb数据集
        self.__wvmodel = gensim.models.KeyedVectors.load_word2vec_format(self.__glove_file)
        for seg in self.__segs:
            self.__parse_imdb_datas(seg)
            self.__parse_features_and_labels(seg)
            self.__gen_weight_np(seg)
    def __parse_imdb_datas(self, seg):
#从原始文本中加载数据
        data_lists = []
        for label_name, label_id in self.__label_dic.items():
            sentence_dir = os.path.join(self.__imdb_path, seg, label_name)
            for file in os.listdir(sentence_dir):
                with open(os.path.join(sentence_dir, file), mode='r', encoding='utf8') as f:
                    sentence = f.read().replace('\n', '')
                    data_lists.append([sentence, label_id])
        self.__imdb_datas[seg] = data_lists
    def __parse_features_and_labels(self, seg):
#解析features与labels
        features = []
        labels = []
        for sentence, label in self.__imdb_datas[seg]:
            features.append(sentence)
            labels.append(label)
        self.__features[seg] = features
        self.__labels[seg] = labels
        self.__updata_features_to_tokenized(seg)
        self.__parse_vacab(seg)
        self.__encode_features(seg)
        self.__padding_features(seg)
    def __updata_features_to_tokenized(self, seg):
#切分原始语句
        tokenized_features = []
        for sentence in self.__features[seg]:
            tokenized_sentence = [word.lower() for word in sentence.split(" ")]
            tokenized_features.append(tokenized_sentence)
        self.__features[seg] = tokenized_features
    def __parse_vacab(self, seg):
#构建词汇表
        tokenized_features = self.__features[seg]
        vocab = set(chain(*tokenized_features))
        self.__vacab[seg] = vocab
        word_to_idx = {word: i + 1 for i, word in enumerate(vocab)}
        word_to_idx['<unk>'] = 0
        self.__word2idx[seg] = word_to_idx
    def __encode_features(self, seg):
#词汇编码
        word_to_idx = self.__word2idx['train']
        encoded_features = []
        for tokenized_sentence in self.__features[seg]:
            encoded_sentence = []
            for word in tokenized_sentence:
                encoded_sentence.append(word_to_idx.get(word, 0))
            encoded_features.append(encoded_sentence)
        self.__features[seg] = encoded_features
    def __padding_features(self, seg, maxlen=500, pad=0):
# 将所有features填充到相同的长度
        padded_features = []
        for feature in self.__features[seg]:
            if len(feature) >= maxlen:
                padded_feature = feature[:maxlen]
            else:
                padded_feature = feature
                while len(padded_feature) < maxlen:
                    padded_feature.append(pad)
            padded_features.append(padded_feature)
        self.__features[seg] = padded_features
    def __gen_weight_np(self, seg):
   # 使用gensim获取权重
        weight_np = np.zeros((len(self.__word2idx[seg]), self.__glove_dim), dtype=np.float32)
        for word, idx in self.__word2idx[seg].items():
            if word not in self.__wvmodel:
                continue
            word_vector = self.__wvmodel.get_vector(word)
            weight_np[idx, :] = word_vector
        self.__weight_np[seg] = weight_np
    def get_datas(self, seg):
#返回 features, labels, weight
        features = np.array(self.__features[seg]).astype(np.int32)
        labels = np.array(self.__labels[seg]).astype(np.int32)
        weight = np.array(self.__weight_np[seg])
        return features, labels, weight
def _convert_to_mindrecord(data_home, features, labels, weight_np=None, training=True):
#将原始数据集转换为mindrecord格式
    if weight_np is not None:
        np.savetxt(os.path.join(data_home, 'weight.txt'), weight_np)
# 写入mindrecord
    schema_json = {"id": {"type": "int32"},
                   "label": {"type": "int32"},
                   "feature": {"type": "int32", "shape": [-1]}}
    data_dir = os.path.join(data_home, "aclImdb_train.mindrecord")
    if not training:
        data_dir = os.path.join(data_home, "aclImdb_test.mindrecord")
    def get_imdb_data(features, labels):
        data_list = []
        for i, (label, feature) in enumerate(zip(labels, features)):
            data_json = {"id": i,
                         "label": int(label),
                         "feature": feature.reshape(-1)}
            data_list.append(data_json)
        return data_list
    writer = FileWriter(data_dir, shard_num=4)
    data = get_imdb_data(features, labels)
    writer.add_schema(schema_json, "nlp_schema")
    writer.add_index(["id", "label"])
    writer.write_raw_data(data)
    writer.commit()
def convert_to_mindrecord(embed_size, aclimdb_path, preprocess_path, glove_path):
#将原始数据集转换为mindrecord格式
    parser = ImdbParser(aclimdb_path, glove_path, embed_size)
    parser.parse()
    if not os.path.exists(preprocess_path):
        print(f"preprocess path {preprocess_path} is not exist")
        os.makedirs(preprocess_path)
    train_features, train_labels, train_weight_np = parser.get_datas('train')
    _convert_to_mindrecord(preprocess_path, train_features, train_labels, train_weight_np)
    test_features, test_labels, _ = parser.get_datas('test')
    _convert_to_mindrecord(preprocess_path, test_features, test_labels, training=False)
if args.preprocess == "true":
    os.system("rm -f ./preprocess/aclImdb* weight*")
    print("============== Starting Data Pre-processing ==============")
    convert_to_mindrecord(cfg.embed_size, args.aclimdb_path, args.preprocess_path, args.glove_path)
    print("======================= Successful =======================")

============== Starting Data Pre-processing ==============
======================= Successful =======================


(4)创建训练集：
定义创建数据集函数lstm_create_dataset，创建训练集ds_train。

通过create_dict_iterator方法创建字典迭代器，读取已创建的数据集ds_train中的数据。

运行以下一段代码，创建数据集并读取第1个batch中的label数据列表和第1个batch中第1个元素的feature数据。

In [8]:
import os
import mindspore.dataset as ds
def lstm_create_dataset(data_home, batch_size, repeat_num=1, training=True):
#创建数据集
    ds.config.set_seed(1)
    data_dir = os.path.join(data_home, "aclImdb_train.mindrecord0")
    if not training:
        data_dir = os.path.join(data_home, "aclImdb_test.mindrecord0")
    data_set = ds.MindDataset(data_dir, columns_list=["feature", "label"], num_parallel_workers=4)
# 对数据集进行shuffle、batch与repeat操作
    data_set = data_set.shuffle(buffer_size=data_set.get_dataset_size())
    data_set = data_set.batch(batch_size=batch_size, drop_remainder=True)
    data_set = data_set.repeat(count=repeat_num)
    return data_set
ds_train = lstm_create_dataset(args.preprocess_path, cfg.batch_size)
iterator = next(ds_train.create_dict_iterator())
first_batch_label = iterator["label"].asnumpy()
first_batch_first_feature = iterator["feature"].asnumpy()[0]
print(f"The first batch contains label below:\n{first_batch_label}\n")
print(f"The feature of the first item in the first batch is below vector:\n{first_batch_first_feature}")

The first batch contains label below:
[0 0 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1
 0 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 0 0 1 1 0 1 1 0]

The feature of the first item in the first batch is below vector:
[249996  54143 184172 203651 229589 221693 185989 118515  64846  54704
  19712 140286  54143  10035 223633 182804 110279  20992 185989 118515
  54143 229589 124426 189682 129826  98619 251411  16315 100038 112995
 237022 116461  30735 229874  38533  25750  44090  30219  30735 229874
 171780 118515  65081  44090  74354 128277  82354 118515 215392  61497
 212639    923 210633 105168 249996  54143 185745 184172 187822 185213
 223619 100038  65443  73067 129442  44090 118515 156542  82301 111804
  66658 184172  42988  95885 185989  76874  13192 171920 229589 156542
  45558   5290  52959  80287  91542  91662 114496 112876  42988 192087
 185507 186212  66658 233582 230976 143758 128277 215027 229589 154143
 246234 167821 184159  40065 100038 112995 238258 1805

(5)模型构建

导入初始化网络所需模块。

定义需要单层LSTM小算子堆叠的设备类型。

定义lstm_default_state函数来初始化网络参数及网络状态。

定义stack_lstm_default_state函数来初始化小算子堆叠需要的初始化网络参数及网络状态。

针对CPU场景，自定义单层LSTM小算子堆叠，来实现多层LSTM大算子功能。

使用Cell方法，定义网络结构（SentimentNet网络）。

实例化SentimentNet，创建网络，最后输出网络中加载的参数。

In [9]:
import math
import numpy as np
from mindspore import Tensor, nn, context, Parameter, ParameterTuple
from mindspore.common.initializer import initializer
import mindspore.ops as ops
# 当设备类型为CPU时采用堆叠类型的LSTM
STACK_LSTM_DEVICE = ["CPU"]
# 将短期记忆(h)和长期记忆(c)初始化为0
def lstm_default_state(batch_size, hidden_size, num_layers, bidirectional):
#LSTM网络输入初始化
    num_directions = 2 if bidirectional else 1
    h = Tensor(np.zeros((num_layers * num_directions, batch_size, hidden_size)).astype(np.float32))
    c = Tensor(np.zeros((num_layers * num_directions, batch_size, hidden_size)).astype(np.float32))
    return h, c
def stack_lstm_default_state(batch_size, hidden_size, num_layers, bidirectional):
#STACK LSTM网络输入初始化
    num_directions = 2 if bidirectional else 1
    h_list = c_list = []
    for _ in range(num_layers):
        h_list.append(Tensor(np.zeros((num_directions, batch_size, hidden_size)).astype(np.float32)))
        c_list.append(Tensor(np.zeros((num_directions, batch_size, hidden_size)).astype(np.float32)))
    h, c = tuple(h_list), tuple(c_list)
    return h, c
class StackLSTM(nn.Cell):
#  实现堆叠LSTM
    def __init__(self,
                 input_size,
                 hidden_size,
                 num_layers=1,
                 has_bias=True,
                 batch_first=False,
                 dropout=0.0,
                 bidirectional=False):
        super(StackLSTM, self).__init__()
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.transpose = ops.Transpose()
        num_directions = 2 if bidirectional else 1
        input_size_list = [input_size]
        for i in range(num_layers - 1):
            input_size_list.append(hidden_size * num_directions)
# LSTMCell为单层RNN结构，通过堆叠LSTMCell可完成StackLSTM
        layers = []
        for i in range(num_layers):
            layers.append(nn.LSTMCell(input_size=input_size_list[i],
                                      hidden_size=hidden_size,
                                      has_bias=has_bias,
                                      batch_first=batch_first,
                                      bidirectional=bidirectional,
                                      dropout=dropout))
# 权重初始化
        weights = []
        for i in range(num_layers):
            weight_size = (input_size_list[i] + hidden_size) * num_directions * hidden_size * 4
            if has_bias:
                bias_size = num_directions * hidden_size * 4
                weight_size = weight_size + bias_size
            stdv = 1 / math.sqrt(hidden_size)
            w_np = np.random.uniform(-stdv, stdv, (weight_size, 1, 1)).astype(np.float32)
            weights.append(Parameter(initializer(Tensor(w_np), w_np.shape), name="weight" + str(i)))
        self.lstms = layers
        self.weight = ParameterTuple(tuple(weights))
    def construct(self, x, hx):
#构建网络
        if self.batch_first:
            x = self.transpose(x, (1, 0, 2))
        h, c = hx
        hn = cn = None
        for i in range(self.num_layers):
            x, hn, cn, _, _ = self.lstms[i](x, h[i], c[i], self.weight[i])
        if self.batch_first:
            x = self.transpose(x, (1, 0, 2))
        return x, (hn, cn)
class SentimentNet(nn.Cell):
#构建SentimentNet
    def __init__(self,
                 vocab_size,
                 embed_size,
                 num_hiddens,
                 num_layers,
                 bidirectional,
                 num_classes,
                 weight,
                 batch_size):
        super(SentimentNet, self).__init__()
# 对数据中的词汇进行降维
        self.embedding = nn.Embedding(vocab_size,
                                      embed_size,
                                      embedding_table=weight)
        self.embedding.embedding_table.requires_grad = False
        self.trans = ops.Transpose()
        self.perm = (1, 0, 2)
# 判断是否需要堆叠LSTM
        if context.get_context("device_target") in STACK_LSTM_DEVICE:
            self.encoder = StackLSTM(input_size=embed_size,
                                     hidden_size=num_hiddens,
                                     num_layers=num_layers,
                                     has_bias=True,
                                     bidirectional=bidirectional,
                                     dropout=0.0)
            self.h, self.c = stack_lstm_default_state(batch_size, num_hiddens, num_layers, bidirectional)
        else:
            self.encoder = nn.LSTM(input_size=embed_size,
                                   hidden_size=num_hiddens,
                                   num_layers=num_layers,
                                   has_bias=True,
                                   bidirectional=bidirectional,
                                   dropout=0.0)
            self.h, self.c = lstm_default_state(batch_size, num_hiddens, num_layers, bidirectional)
        self.concat = ops.Concat(1)
        if bidirectional:
            self.decoder = nn.Dense(num_hiddens * 4, num_classes)
        else:
            self.decoder = nn.Dense(num_hiddens * 2, num_classes)
    def construct(self, inputs):
# 输入：(64,500,300)
        embeddings = self.embedding(inputs)
        embeddings = self.trans(embeddings, self.perm)
        output, _ = self.encoder(embeddings, (self.h, self.c))
        encoding = self.concat((output[0], output[499]))
        outputs = self.decoder(encoding)
        return outputs
embedding_table = np.loadtxt(os.path.join(args.preprocess_path, "weight.txt")).astype(np.float32)
network = SentimentNet(vocab_size=embedding_table.shape[0],
                       embed_size=cfg.embed_size,
                       num_hiddens=cfg.num_hiddens,
                       num_layers=cfg.num_layers,
                       bidirectional=cfg.bidirectional,
                       num_classes=cfg.num_classes,
                       weight=Tensor(embedding_table),
                       batch_size=cfg.batch_size)
print(network.parameters_dict(recurse=True))

OrderedDict([('embedding.embedding_table', Parameter (name=embedding.embedding_table, value=Tensor(shape=[252193, 300], dtype=Float32, value=
[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 ...
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 [-2.64310002e-01,  2.03539997e-01, -1.07670002e-01 ...  3.17510009e-01, -6.45749986e-01,  4.42129999e-01],
 [-2.82150000e-01,  2.53950000e-01,  3.94300014e-01 ...  1.75999999e-01,  7.86110014e-02, -7.89420009e-02]]))), ('encoder.weight', Parameter (name=encoder.weight, value=Tensor(shape=[563200, 1, 1], dtype=Float32, value=
[[[-1.65955983e-02]],
 [[ 4.40648980e-02]],
 [[-9.99771282e-02]],
 ...
 [[-6.54547513e-02]],


## 6 模型训练
根据建立的LSTM模型，对模型进行训练：

运行以下一段代码，创建优化器和损失函数模型，加载训练数据集（ds_train）并配置好CheckPoint生成信息，然后使用model.train接口，进行模型训练。根据输出可以看到loss值随着训练逐步降低，最后达到0.262左右。

In [10]:
from mindspore import Model
from mindspore.train.callback import CheckpointConfig, ModelCheckpoint, TimeMonitor, LossMonitor
from mindspore.nn import Accuracy
from mindspore import nn
os.system("rm -f {0}/*.ckpt {0}/*.meta".format(args.ckpt_path))
#定义损失函数
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
opt = nn.Momentum(network.trainable_params(), cfg.learning_rate, cfg.momentum)
#创建模型对象 model
model = Model(network, loss, opt, {'acc': Accuracy()})
#创建损失监视器（LossMonitor）对象
loss_cb = LossMonitor(per_print_times=78)
#输出训练开始的提示信息
print("============== Starting Training ==============")
#配置模型的检查点（checkpoint）保存方式
config_ck = CheckpointConfig(save_checkpoint_steps=cfg.save_checkpoint_steps,
                             keep_checkpoint_max=cfg.keep_checkpoint_max)
ckpoint_cb = ModelCheckpoint(prefix="lstm", directory=args.ckpt_path, config=config_ck)
#创建时间监视器（TimeMonitor）对象
time_cb = TimeMonitor(data_size=ds_train.get_dataset_size())
if args.device_target == "CPU":
    model.train(cfg.num_epochs, ds_train, callbacks=[time_cb, ckpoint_cb, loss_cb], dataset_sink_mode=False)
else:
    model.train(cfg.num_epochs, ds_train, callbacks=[time_cb, ckpoint_cb, loss_cb])
print("============== Training Success ==============")

============== Starting Training ==============
epoch: 1 step: 78, loss is 0.2971678
epoch: 1 step: 156, loss is 0.30519545
epoch: 1 step: 234, loss is 0.2370582
epoch: 1 step: 312, loss is 0.25823578
epoch: 1 step: 390, loss is 0.2899053
Epoch time: 27745.798, per step time: 71.143
epoch: 2 step: 78, loss is 0.20885809
epoch: 2 step: 156, loss is 0.2168142
epoch: 2 step: 234, loss is 0.14624771
epoch: 2 step: 312, loss is 0.2152691
epoch: 2 step: 390, loss is 0.3756763
Epoch time: 27407.312, per step time: 70.275
epoch: 3 step: 78, loss is 0.116764486
epoch: 3 step: 156, loss is 0.20790516
epoch: 3 step: 234, loss is 0.2118046
epoch: 3 step: 312, loss is 0.18587393
epoch: 3 step: 390, loss is 0.25241128
Epoch time: 27251.069, per step time: 69.875
epoch: 4 step: 78, loss is 0.11729147
epoch: 4 step: 156, loss is 0.16071466
epoch: 4 step: 234, loss is 0.43869072
epoch: 4 step: 312, loss is 0.37149796
epoch: 4 step: 390, loss is 0.18670222
Epoch time: 27441.597, per step time: 70.363
ep

## 7 模型测试
根据处理后的测试数据以及建立的LSTM模型，对模型进行测试：

创建并加载验证数据集（ds_eval），加载由训练保存的CheckPoint文件，进行验证，查看模型质量，此步骤用时约30秒。

In [11]:
from mindspore import load_checkpoint, load_param_into_net
#加载模型的检查点文件和将参数加载到网络中
args.ckpt_path_saved = f'{args.ckpt_path}/lstm-{cfg.num_epochs}_390.ckpt'
print("============== Starting Testing ==============")
ds_eval = lstm_create_dataset(args.preprocess_path, cfg.batch_size, training=False)
#加载之前保存的检查点文件args.ckpt_path_saved，并将参数加载到网络network中
param_dict = load_checkpoint(args.ckpt_path_saved)
load_param_into_net(network, param_dict)
if args.device_target == "CPU":
    acc = model.eval(ds_eval, dataset_sink_mode=False)
else:
    acc = model.eval(ds_eval)
#输出准确性
print("============== {} ==============".format(acc))

============== Starting Testing ==============
============== {'acc': 0.8476362179487179} ==============


## 8 实验总结

以上便完成了MindSpore自然语言处理应用的体验，通过本次体验全面了解了如何使用MindSpore进行自然语言中处理情感分类问题，理解了如何通过定义和初始化基于LSTM的SentimentNet网络进行训练模型及验证正确率。